<a href="https://colab.research.google.com/github/EsraaAdinar/python-and-cyber/blob/main/Lamda_interface_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 9.2 MB/s eta 0:00:00


In [2]:
import os
from IPython.display import clear_output

def set_environment_variable_values():
  ACCESS_KEY =input("Please enter the AWS access key: ")
  SECRET_ACCESS_KEY =input("Please enter the AWS secret access key: ")
  BUCKET_NAME =input("Please enter the name of the bucket in S3: ")
  os.environ['ACCESS_KEY'] = ACCESS_KEY
  os.environ['SECRET_ACCESS_KEY'] = SECRET_ACCESS_KEY
  os.environ['BUCKET_NAME'] = BUCKET_NAME
  clear_output()
  return None

set_environment_variable_values()

In [3]:
import os
import boto3

def get_S3_client():
    resource = boto3.client(
        "s3",
        aws_access_key_id=os.environ.get('ACCESS_KEY'),
        aws_secret_access_key=os.environ.get('SECRET_ACCESS_KEY')
    )
    return resource


# Get the list of schools from S3


In [ ]:
def get_file(filename):
    try:
        # get the file from the bucket
        s3_client = get_S3_client()
        file_object = s3_client.get_object(Bucket=os.environ.get('BUCKET_NAME'), Key=filename)

        # read the data from the file object and decode it from bytes data to string data
        data_file = file_object['Body'].read()
        return str(data_file.decode("utf-8"))
    except Exception as e:
        print(f"Error fetching file '{filename}': {str(e)}")
        return None


data = get_file('schools_list.json')
if data:
    print(data)

else:
    print("Failed to retrieve data from file.")


# Store list into Lamda

In [43]:
def save_schools_to_s3(schools_list, s3_bucket_name, s3_key):
    try:
        # Get S3 client
        s3_client = get_S3_client()

        if schools_list:
             # Convert list of dictionaries to JSON format
            json_data = json.dumps(schools_list, indent=2)

            # Upload JSON file to S3
            s3_client.put_object(
                Bucket=s3_bucket_name,
                Key=s3_key,
                Body=json_data,
                ContentType='application/json'
            )
            print("List of schools saved to S3 at s3 as json.")
        else:
            print("Error: The list of schools is empty.No list saved at S3")
    except Exception as e:
        print(f"Error: {str(e)}")



# 8-Lamda Function

In [37]:
import os
import json
#from functions import get_S3_client, get_file

def lambda_handler(event, context):
    try:
        request_body = event.get('body', {})
        action = request_body.get('data', {}).get('action')
        latitude = request_body.get('data', {}).get('latitude')
        longitude = request_body.get('data', {}).get('longitude')
        radius = request_body.get('data', {}).get('radius')
        datalist = request_body.get('data', {}).get('datalist')
        csv_filename= request_body.get('data',{}).get('csv_filename')

        if "body" in event.keys():
            request = event["body"]
            if type(request) is not dict:
                request = json.loads(request)

            if request is not None and "data" in request.keys():
                data = request["data"]
                operation = data['action']
                if operation == "store":
                    s3_bucket_name = os.environ.get('BUCKET_NAME')
                    s3_key = 'school_list.csv'
                    save_schools_to_s3(datalist, s3_bucket_name, s3_key)
                    return_data=datalist
                    return {
                        "statusCode": 200,
                        "body": json.dumps(return_data)
                    }
                elif operation == "get":
                   return_data = get_file(csv_filename)
                   return {
                        "statusCode": 200,
                        "headers":{
                        "Content-Type" : "application/json",
                        "Access-Control-Allow-Headers" : "Content-Type,X-Api-Key",
                        "Access-Control-Allow-Methods" : "POST",
                        "Access-Control-Allow-Origin":"*"
                            },
                       "body": json.dumps(return_data),
                    }
                else:
                    return {
                        "statusCode": 400,
                        "body": json.dumps({"error": "Invalid filter keyword"})
                    }
    except Exception as e:
        return {
            "statusCode": 404,
            "headers": {
                "Content-Type": "application/json",
                "Access-Control-Allow-Headers": "Content-Type,X-Api-Key",
                "Access-Control-Allow-Methods": "POST",
                "Access-Control-Allow-Origin": "*"
            },
            "body": f"there was an error, {e}"
        }


# Test Events

In [51]:
request_event = {

     "body":  {
         "data": {
             "action":"store",
             "latitude":99.99999 ,
             "longitude":0 ,
             "radius":10,
             "datalist":[]
         }
     }
}
   # Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

Error: The list of schools is empty.No list saved at S3
Status code is: 200
Result is: []


In [44]:
request_event = {

     "body":  {
         "data": {
             "action":"store",
             "latitude":55.816555  ,
             "longitude":-4.309890  ,
             "radius":4,
             "datalist":["Shawlands Primary School",
    "Holy Cross Primary School",
    "Sandwood Primary School",
    "Hillington Primary School",
    "Our Lady of the Mission Primary School",
    "Cuthbertson Primary School",
    "Ashpark Primary School",
    "Merrylee Primary School",
    "Cardonald Primary School",
    "Pollokshields Primary School",
    "Rosshall Academy",
    "Hollybrook Academy",
    "Mosspark Primary School",
    "Bellahouston Academy",
    "Crookfur Primary School",
    "King's Park Primary School",
    "Craigton Primary School",
    "King's Park Secondary School",
    "Our Lady of the Rosary Primary School",
    "Carolside Primary School"
             ]
         }
     }
}
   # Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

List of schools saved to S3 at s3 as json.
Status code is: 200
Result is: ["Shawlands Primary School", "Holy Cross Primary School", "Sandwood Primary School", "Hillington Primary School", "Our Lady of the Mission Primary School", "Cuthbertson Primary School", "Ashpark Primary School", "Merrylee Primary School", "Cardonald Primary School", "Pollokshields Primary School", "Rosshall Academy", "Hollybrook Academy", "Mosspark Primary School", "Bellahouston Academy", "Crookfur Primary School", "King's Park Primary School", "Craigton Primary School", "King's Park Secondary School", "Our Lady of the Rosary Primary School", "Carolside Primary School"]


In [45]:
request_event = {

     "body":  {
         "data": {
              "action": "store",
              "latitude": 87.543255,
              "longitude": 0.0,
              "radius": 10.0,
              "datalist": []
         }
     }
}
   # Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

Error: The list of schools is empty.No list saved at S3
Status code is: 200
Result is: []


In [16]:
request_event = {

     "body":  {
         "data": {

         }
     }
}
   # Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

Status code is: 404
Result is: there was an error, 'action'


In [17]:
request_event = {

     "body":  {
         "data": {
             "action":"give",
             "latitude":87.543255 ,
             "longitude":0  ,
             "radius":10,
             "datalist":"schools_list"
         }
     }
}
   # Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

Status code is: 400
Result is: {"error": "Invalid filter keyword"}


In [49]:
## Set up the data
request_event = {

    "body": {
        "data": {
            "action":"get",
            "csv_filename": "schools_list.csv"
        }
    }
}#get the file inside the lamda function



# Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

Status code is: 200
Result is: "[\n  {\n    \"Name\": \"Shawlands Primary School\",\n    \"Latitude\": 55.82952239999999,\n    \"Longitude\": -4.2855491,\n    \"Radius\": 4\n  },\n  {\n    \"Name\": \"Sir John Maxwell School\",\n    \"Latitude\": 55.8255717,\n    \"Longitude\": -4.2970815,\n    \"Radius\": 4\n  },\n  {\n    \"Name\": \"Hillington Primary School\",\n    \"Latitude\": 55.8524158,\n    \"Longitude\": -4.3582729,\n    \"Radius\": 4\n  },\n  {\n    \"Name\": \"Pollokshields Primary School\",\n    \"Latitude\": 55.84229389999999,\n    \"Longitude\": -4.2747136,\n    \"Radius\": 4\n  },\n  {\n    \"Name\": \"Sandwood Primary School\",\n    \"Latitude\": 55.8495065,\n    \"Longitude\": -4.3621152,\n    \"Radius\": 4\n  },\n  {\n    \"Name\": \"Cuthbertson Primary School\",\n    \"Latitude\": 55.8397751,\n    \"Longitude\": -4.2651444,\n    \"Radius\": 4\n  },\n  {\n    \"Name\": \"Ibrox Primary School & Nursery Class\",\n    \"Latitude\": 55.8517114,\n    \"Longitude\": -4.308